# LSTM

## Importing the modules

In [78]:
import torch
import torch.nn as nn
import torchtext

from torch.utils.data import Dataset
from torch.utils.data import DataLoader


from torchtext.data import Field
from torchtext.data import TabularDataset

##  Constructing the Dataset

In [92]:
import random



class FalseToxicDataset(Dataset):
    
    def __init__(self):
        pass
    
    def __len__(self):
        return 5

    def __getitem__(self, index):
        return torch.tensor([random.randint(0,120) for i in range(10)],dtype=torch.long)
    
    
    
        
        

In [93]:
train = FalseToxicDataset()
BATCH_SIZE =1
data_loader = DataLoader(train,batch_size = BATCH_SIZE, pin_memory=True)




## Defining our model

In [94]:
# Here we define our model as a class
class BiLSTM(nn.Module):

    def __init__(self, input_dim=10, hidden_dim = 250, batch_size=1, output_dim=6, num_layers=1):
        
        super(BiLSTM, self).__init__()
        
        # Attributes
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer (bidirectionnal)
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, bidirectional=True)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)



In [95]:
model = BiLSTM()

## Definition of our loss fonction

In [96]:
criterion = torch.nn.MSELoss()

## Definition of our optimizer

In [97]:
optimiser = torch.optim.Adam(model.parameters())

## Train phase

In [100]:

for epoch in range(5):
    for batch in data_loader:
        # Clear stored gradient
        model.zero_grad()

        # Initialise hidden state
        # Don't do this if you want your LSTM to be stateful
        model.hidden = model.init_hidden()

        # Forward pass
        y_pred = model(batch)

        loss = criterion (y_pred, torch.tensor([random.random() for i in range(6)],dtype=torch.long))
        

        # Zero out gradient, else they will accumulate between epochs
        optimiser.zero_grad()

        # Backward pass
        loss.backward()

        # Update parameters
        optimiser.step()

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'mat2'